In [3]:
#Simulate RPC
from rdflib import RDFS, RDF, Namespace, Graph, URIRef
from sssontology import sssontology
import random
ssso = Namespace('http://www.semanticweb.org/linch/ontologies/2018/11/ssso#')
SmartConferenceRoom=sssontology('ssso_v1.ttl')

ImportError: No module named sssontology

In [2]:
def sl_update(obj,au,sl):
    try:
        prev_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:has ?sl.}"""%obj)[0][0].split(':')[-1]
        prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ssso:%s ssso:isProvidedBy ?au.}"""%obj)
    except:
        print("sl_update_error")
        return
    label=False
    for i in range(len(prev_auth)):
        if(au in prev_auth[i][0]):
            label=True
    if(not label):
        SmartConferenceRoom.remove((ssso[obj],ssso['has'],ssso[prev_sl]))
        new_sl=abs(int(prev_sl[-1])+int(sl[-1]))
        if(new_sl>4):
            new_sl=4
        new_sl='SL-'+str(new_sl)
        SmartConferenceRoom.add((ssso[obj],ssso['has'],ssso[new_sl]))
        SmartConferenceRoom.add((ssso[obj],ssso['isProvidedBy'],ssso[au]))
        print('sl_update: %s:%s->%s'%(obj,prev_sl,new_sl))
        SmartConferenceRoom.commit()
        SmartConferenceRoom.refresh()
    else:
        print('Repeated Authentication: Current sl: %s'%(prev_sl))
def sl_diff(sl1,sl2):
    return('SL-'+str(abs(int(sl1[-1])-int(sl2[-1]))))
def max_tl(tl_list):
    if(['ssso:TL-High']in tl_list):
        return ('TL-High')
    elif(['ssso:TL-Mid']in tl_list):
        return ('TL-Mid')
    elif(['ssso:TL-Low']in tl_list):
        return('TL-Low')
    else:
        return('TL-None')
def sldegradation(sl,tl):
    if(tl=='TL-High'):
        sl_value=int(sl[-1])-4
    elif(tl=='TL-Mid'):
        sl_value=int(sl[-1])-2
    elif(tl=='TL-Low'):
        sl_value=int(sl[-1])-1
    elif(tl=='TL-None'):
        sl_value=int(sl[-1])
        return('SL-'+str(sl_value))
    if(sl_value<0):
        sl_value=0
    print('Security Level degrades to %s'%('SL-'+str(sl_value)))
    return('SL-'+str(sl_value))
    
def auth(usr,device,sl):
    sl_satisfied_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s. }"""%('SL-2'))
    usr_prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ssso:%s ssso:isProvidedBy ?au.}"""%usr)
    current_device_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s. ssso:%s ssso:provides ?au. ssso:%s ssso:is ssso:Enabled.}"""%(sl,device,device))
    #other_device_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s. ssso:%s ssso:provides ?au. ssso:%s ssso:is ssso:Enabled.}"""%(sl,device,device))
    for i in range(len(current_device_auth)):
        for j in range(len(usr_prev_auth)):
            if(usr_prev_auth[j][0]==current_device_auth[i][0]):
                current_device_auth[i]=['Invalid']
    try:
        current_device_auth.remove(['Invalid'])
    except:
        1
    for i in range(len(sl_satisfied_auth)):
        for j in range(len(usr_prev_auth)):
            if(usr_prev_auth[j][0]==sl_satisfied_auth[i][0]):
                sl_satisfied_auth[i]=['Invalid']
    try:
        sl_satisfied_auth.remove(['Invalid'])
    except:
        1
    if(len(current_device_auth)!=0):
        print('Please use %s to perform %s'%(device,random.sample(current_device_auth,1)[0][0].split(':')[-1]))
    else:
        au=random.sample(sl_satisfied_auth,1)[0][0].split(':')[-1]
        dev=SmartConferenceRoom.query(PREFIX+"""select ?dev where{?dev ssso:is ssso:Enabled. ?dev ssso:provides ssso:%s.}"""%(au))
        print('Please use %s to perform %s'%(random.sample(dev,1)[0][0].split(':')[-1],au))
def sl_runtime_check():
    try:
        roleofroom=SmartConferenceRoom.query(PREFIX+"""select ?rr where {ssso:RoleofRoom ssso:has ?rr.}""")
        print(roleofroom)
        if(len(roleofroom)==0):
            return
    except:
        return
    SmartConferenceRoom.refresh()
    active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv rdf:type ssso:Data. ?serv ssso:is ssso:Active.}""")
    suspended_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv ssso:is ssso:Suspended.}""")
    current_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{ssso:CurrentSecurityLevel ssso:has ?sl.}""")[0][0].split(':')[-1]
    tl_list=SmartConferenceRoom.query(PREFIX+"""select ?tl where {?tr rdf:type ssso:Threat. ?tl rdf:type ssso:ThreatLevel.?tr ssso:is ssso:Active. ?tr ssso:has ?tl}""")
    current_tl=max_tl(tl_list)
    SmartConferenceRoom.remove((ssso['CurrentSecurityLevel'], ssso['has'], ssso[current_sl]))
    current_sl=sldegradation(current_sl,current_tl)
    SmartConferenceRoom.add((ssso['CurrentSecurityLevel'], ssso['has'], ssso[current_sl]))
    SmartConferenceRoom.commit()
    SmartConferenceRoom.refresh()
    for i in range(len(active_service)):
        servicename=active_service[i][0]
        required_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. %s ssso:requires ?sl.}"""%servicename)[0][0].split(':')[-1]
        if(current_sl<required_sl): #for future use, consider user sl
            SmartConferenceRoom.remove((ssso[servicename.split(':')[-1]],ssso['is'],ssso['Active']))
            SmartConferenceRoom.add((ssso[servicename.split(':')[-1]],ssso['is'],ssso['Suspended']))
            print("%s is suspended, since the current security level is not satisified."%servicename.split(':')[-1])
    for i in range(len(suspended_service)):
        servicename=suspended_service[i][0]
        required_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. %s ssso:requires ?sl.}"""%servicename)[0][0].split(':')[-1]
        if(current_sl>=required_sl): #for future use, consider user sl
            SmartConferenceRoom.remove((ssso[servicename.split(':')[-1]],ssso['is'],ssso['Suspended']))
            SmartConferenceRoom.add((ssso[servicename.split(':')[-1]],ssso['is'],ssso['Active']))
            print("%s is resumed, since the threat is mitigated and current security level is satisified."%servicename.split(':')[-1])  
    SmartConferenceRoom.commit()
    SmartConferenceRoom.refresh()

In [3]:
PREFIX="\nPREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>\nPREFIX ssso: <http://www.semanticweb.org/linch/ontologies/2018/11/ssso#>\nPREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>\n"
def report_event(data):
    SmartConferenceRoom.refresh()
    try:
        current_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{ssso:CurrentSecurityLevel ssso:has ?sl.}""")[0][0].split(':')[-1]
    except:
        current_sl=-1
        #return
    event_name=data["event"]["EventName"]
    event_type=data["event"]["EventType"]
    event_status=data["event"]["EventStatus"]
    event_timestamp=data["event"]["EventTimestamp"]
    event_id=data["event"]["EventID"]
    event_message=data["event"]["EventMessage"]
    device_name=data["device"]["DeviceName"]
    if(event_type=="Threat" and event_status=="Active"):
        ql="""select ?t where{ssso:%s ssso:is ?t.}"""%event_name
        res=SmartConferenceRoom.query(PREFIX+ql)
        if(res==[]):
            print("Unknown threat: Would you like to add this threat into knowledge database?")
        elif(res[0][0]=="ssso:Template"):
            tl=SmartConferenceRoom.query(PREFIX+"""select ?tl where{?tl rdf:type ssso:ThreatLevel. ssso:%s ssso:has ?tl.}"""%event_name)[0][0].split(':')[-1]
            SmartConferenceRoom.add((ssso[event_name+'_'+event_id], RDF.type, ssso['Threat'])) #Add Threat Instance
            SmartConferenceRoom.add((ssso[event_name+'_'+event_id], ssso['has'], ssso[tl])) #Add ThreatLevel according to the template
            SmartConferenceRoom.add((ssso[event_name+'_'+event_id], ssso['isProvidedBy'], ssso[device_name])) #Add Device Information
            #SmartConferenceRoom.add((ssso[event_name+'_'+event_id], ssso['influences'], ssso[device_name])) #For future use (Influence)
            SmartConferenceRoom.add((ssso[event_name+'_'+event_id], ssso['is'], ssso['Active'])) #Status: Active Threat
            print("New Threat: "+event_name+'_'+event_id+" ThreatLevel: "+tl)
            SmartConferenceRoom.commit()
            SmartConferenceRoom.refresh()
            if(current_sl!=-1):
                sl_runtime_check()
    elif(event_type=="Threat" and "Mitigated" in event_status):
        ql="""select ?t where{ssso:%s ssso:is ?t.}"""%(event_name+"_"+event_status.split(":")[1])
        res=SmartConferenceRoom.query(PREFIX+ql)
        if(res==[]):
            print("Unknown threat event: Would you like to manually remove threats?")
        elif(res[0][0]=="ssso:Active"):
            SmartConferenceRoom.remove((ssso[event_name+"_"+event_status.split(":")[1]],None,None))
            print("Threat Removed: "+event_name+'_'+event_status.split(":")[1])
            SmartConferenceRoom.commit()
            SmartConferenceRoom.refresh()
            if(current_sl!=-1):
                sl_runtime_check()
        else:
            print("Inactive threat: Would you like to manually remove threats?")
    if(event_type=="Authentication"):
        if(not("User" in data)):
            print("Invalid Authentication")
            return
        username=data['User']['UserName']
        uid=data['User']['UID']
        name=SmartConferenceRoom.query((PREFIX+"""select ?n where{?n rdf:type ssso:User. ?n ssso:has ssso:%s .}"""%uid))[0][0].split(':')[-1]
        if(name!=username):
            print("Uninvited User")
            return
        ql="""select ?t where{?t rdf:type ssso:Status. ssso:%s ssso:is ?t.}"""%irisauth_alice["event"]["EventName"]
        res=SmartConferenceRoom.query(PREFIX+ql)
        if(res==[]):
            print("Invalid Authentication")
        elif(res[0][0]=="ssso:Enabled"):
            sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:provides ?sl.}"""%event_name)[0][0].split(':')[-1]
            #usl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:has ?sl.}"""%username)[0][0].split(':')[-1]
            sl_update(username,event_name,sl)
        else:
            print("Invalid Authentication")
    if(event_type=="Request"):
        ql="""select ?st where{?st rdf:type ssso:Status. ssso:%s ssso:is ?st.}"""%device_name
        res=SmartConferenceRoom.query(PREFIX+ql)
        if(res==[]):
            print("Disabled Device: Request Denied")
            return
        if(res[0][0].split(':')[-1]!="Enabled"):
            print("Disabled Device: Request Denied")
            return
        servicename=data['event']['EventName']
        ql="""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:requires ?sl.}"""%servicename
        res=SmartConferenceRoom.query(PREFIX+ql)
        if(res==[]):
            print("Unknown Service: Request Denied") #Since roleofroom=classified
            return
        required_sl=res[0][0].split(':')[-1]
        service_category=SmartConferenceRoom.query(PREFIX+"""select ?sc where{?sc rdfs:subClassOf ssso:Service. ssso:%s rdf:type ?sc.}"""%servicename)
        if((not("User" in data)and(current_sl>=required_sl))):
            print("Request Approved: %s"%servicename)
            if(not(['ssso:Control'] in service_category or ['ssso:Authentication'] in service_category)):
                SmartConferenceRoom.add((ssso[event_name], ssso['is'], ssso['Active']))
                print("%s is activated."%servicename)
                SmartConferenceRoom.commit()
                SmartConferenceRoom.refresh()
            return
        elif((not("User" in data)and(current_sl<required_sl))):
            print("Request Denied: Low SL current_sl:%s required_sl:%s"%(current_sl,required_sl))
            sl_df=sl_diff(current_sl,required_sl)
            return
        if((("User" in data)and(current_sl>=required_sl)and(SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:has ?sl.}"""%data['User']['UserName'])[0][0].split(':')[-1]>=required_sl))):
            print("Request Approved: %s"%servicename)
            if(not(['ssso:Control'] in service_category or ['ssso:Authentication'] in service_category)):
                SmartConferenceRoom.add((ssso[event_name], ssso['is'], ssso['Active']))
                print("%s is activated."%servicename)
                SmartConferenceRoom.commit()
                SmartConferenceRoom.refresh()
        else:
            username=data['User']['UserName']
            usr_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:has ?sl.}"""%data['User']['UserName'])[0][0].split(':')[-1]
            print("Request Denied: Low SL current_sl:%s user_sl:%s required_sl:%s"%(current_sl,usr_sl,required_sl))
            sl_df=max(sl_diff(current_sl,required_sl),sl_diff(usr_sl,required_sl))
            print(sl_df+' authentication is required.')
            auth(username,device_name,sl_df)
            return
            

In [4]:
occupancy_camera_event={
    "guid": "95a9c559-9b43-41b5-a270-325e04131735", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T21:23:42.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"82b69dc9-cdbb-4637-ac7a-2141b4fee214",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:43.681662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Threat", # conveys criticality or broad class of the event
	"EventStatus":"Active",#Active / mitigated
	"EventName":"Unexpected_Occupancy", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Unexpected_Occupancy:Environment:Compromised" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"e672ea83-2fb9-4f92-8cc8-d14f068b7d93", # globally unique, typically hash
	"DeviceName":"Camera", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", # Samsung Galaxy 204, iPhone XXVI, etc.
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected", "Brightness": "80", "connection_strength": 12.013}
    }}

In [5]:
report_event(occupancy_camera_event)

New Threat: Unexpected_Occupancy_82b69dc9-cdbb-4637-ac7a-2141b4fee214 ThreatLevel: TL-High


In [6]:
occupancy_light_event={
    "guid": "90cf4586-c3f5-4733-bbcf-95f833f2d321", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T21:23:40.431498+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"98903ea9-9529-476a-a1c8-cb2c57f8fb7a",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:39.123477+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Threat", # conveys criticality or broad class of the event
	"EventStatus":"Active", #Active / mitigated
	"EventName":"Unexpected_Occupancy", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Unexpected_Occupancy:Environment:Compromised" # for future use, not generated yet (description of event)
    },
    "device":
    {
	"DeviceID":"67724eba-754b-4e52-8d39-cf2092c1b049", # globally unique, typically hash
	"DeviceName":"Dimmable_Lights", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lighting", # Samsung Galaxy 204, iPhone XXVI, etc.
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected", "Brightness": "40", "connection_strength": 14.013}
    }}

In [7]:
report_event(occupancy_light_event)

New Threat: Unexpected_Occupancy_98903ea9-9529-476a-a1c8-cb2c57f8fb7a ThreatLevel: TL-High


In [8]:
occupancy_camera_event_mitigated={
    "guid": "b6c5e720-59f5-4f67-8a29-4728d0d09bf7", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T21:23:52.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"826a3c5b-055c-41cb-9e88-94fa8f1df853",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:52.181662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Threat", # conveys criticality or broad class of the event
	"EventStatus":"Mitigated:82b69dc9-cdbb-4637-ac7a-2141b4fee214", #Active / Mitigated
	"EventName":"Unexpected_Occupancy", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Unexpected_Occupancy:Environment:Compromised" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"e672ea83-2fb9-4f92-8cc8-d14f068b7d93", # globally unique, typically hash
	"DeviceName":"Camera", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", # Samsung Galaxy 204, iPhone XXVI, etc.
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected", "Brightness": "80", "connection_strength": 12.013}
    }}

In [9]:
report_event(occupancy_camera_event_mitigated)

Threat Removed: Unexpected_Occupancy_82b69dc9-cdbb-4637-ac7a-2141b4fee214


In [10]:
occupancy_light_event_mitigated={
    "guid": "e8947097-b6da-4456-810b-8eacafa33ae5", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T21:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"5c98b604-8f18-45f5-966c-708c7ff65cf2",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:52.581662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Threat", # conveys criticality or broad class of the event
	"EventStatus":"Mitigated:98903ea9-9529-476a-a1c8-cb2c57f8fb7a", #Active / Mitigated
	"EventName":"Unexpected_Occupancy", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Unexpected_Occupancy:Environment:Compromised" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"e672ea83-2fb9-4f92-8cc8-d14f068b7d93", # globally unique, typically hash
	"DeviceName":"Camera", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", # Samsung Galaxy 204, iPhone XXVI, etc.
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected", "Brightness": "80", "connection_strength": 12.013}
    }}

In [11]:
report_event(occupancy_light_event_mitigated)

Threat Removed: Unexpected_Occupancy_98903ea9-9529-476a-a1c8-cb2c57f8fb7a


In [12]:
irisauth_alice={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"Iris_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [13]:
report_event(irisauth_alice)

sl_update: Alice:SL-0->SL-4


In [14]:
request_open_door_alice={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"OpenDoor", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:OpenDoor" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [15]:
report_event(request_open_door_alice)

Request Approved: OpenDoor


In [16]:
faceauth_bob={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"Face_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Bob",
            "UID":"10002"
           }
}

In [17]:
report_event(faceauth_bob)

sl_update: Bob:SL-0->SL-3


In [18]:
faceauth_bob={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"Face_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Bob",
            "UID":"10002"
           }
}

In [19]:
report_event(faceauth_bob)

Repeated Authentication: Current sl: SL-3


In [20]:
request_open_door_bob={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"OpenDoor", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:OpenDoor" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Bob",
            "UID":"10002"
           }
}

In [21]:
report_event(request_open_door_bob)

Request Denied: Low SL current_sl:SL-4 user_sl:SL-3 required_sl:SL-4
SL-1 authentication is required.
Please use SmartDoorLock to perform NFC_Authentication


In [22]:
nfcauth_bob={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"NFC_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Bob",
            "UID":"10002"
           }
}

In [23]:
report_event(nfcauth_bob)

sl_update: Bob:SL-3->SL-4


In [24]:
report_event(request_open_door_bob)

Request Approved: OpenDoor


In [25]:
passauth_carol={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"Password_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Carol",
            "UID":"10003"
           }
}

In [26]:
report_event(passauth_carol)

sl_update: Carol:SL-0->SL-1


In [27]:
request_open_door_carol={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"OpenDoor", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:OpenDoor" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Carol",
            "UID":"10003"
           }
}

In [28]:
report_event(request_open_door_carol)

Request Denied: Low SL current_sl:SL-4 user_sl:SL-1 required_sl:SL-4
SL-3 authentication is required.
Please use SmartDoorLock to perform Face_Authentication


In [29]:
faceauth_carol={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"Face_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Carol",
            "UID":"10003"
           }
}

In [30]:
report_event(faceauth_carol)

sl_update: Carol:SL-1->SL-4


In [31]:
report_event(request_open_door_carol)

Request Approved: OpenDoor


In [ ]:
NFCauth_carol={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"NFC_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Carol",
            "UID":"10003"
           }
}

In [ ]:
report_event(NFCauth_carol)

In [ ]:
report_event(request_open_door_carol)

In [32]:
use_smart_speaker_to_record_meeting={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"Record", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:Record" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartSpeaker", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [33]:
report_event(use_smart_speaker_to_record_meeting)

Request Approved: Record
Record is activated.


In [34]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:Record']]

In [35]:
use_smart_speaker_to_play_audio={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"PlayAudio", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:PlayAudio" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartSpeaker", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [36]:
report_event(use_smart_speaker_to_play_audio)

Request Approved: PlayAudio
PlayAudio is activated.


In [37]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:PlayAudio'], ['ssso:Record']]

In [38]:
use_camera_to_capture_video={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"CaptureVideo", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:CaptureVideo" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"Camera", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [39]:
report_event(use_camera_to_capture_video)

Request Approved: CaptureVideo
CaptureVideo is activated.


In [40]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:CaptureVideo'], ['ssso:PlayAudio'], ['ssso:Record']]

In [41]:
use_projector_to_play_video={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"PlayVideo", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:PlayVideo" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea3", # globally unique, typically hash
	"DeviceName":"Projector", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [42]:
report_event(use_projector_to_play_video)

Request Approved: PlayVideo
PlayVideo is activated.


In [43]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:PlayVideo'],
 ['ssso:Record'],
 ['ssso:PlayAudio'],
 ['ssso:CaptureVideo']]

In [44]:
use_fileserver_to_store_data={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"StoreData", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:StoreData" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea3", # globally unique, typically hash
	"DeviceName":"FileServer", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Network", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Carol",
            "UID":"10003"
           }
}
use_fileserver_to_share_data={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"ShareData", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:ShareData" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea3", # globally unique, typically hash
	"DeviceName":"FileServer", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Network", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Carol",
            "UID":"10003"
           }
}

In [45]:
report_event(use_fileserver_to_store_data)
report_event(use_fileserver_to_share_data)

Request Approved: StoreData
StoreData is activated.
Request Approved: ShareData
ShareData is activated.


In [46]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:PlayVideo'],
 ['ssso:StoreData'],
 ['ssso:PlayAudio'],
 ['ssso:CaptureVideo'],
 ['ssso:Record'],
 ['ssso:ShareData']]

In [ ]:
#report_threat

In [47]:
unexpected_connection_event={
    "guid": "95a9c559-9b43-41b5-a270-325e04131735", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T21:23:42.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"345b4180-6ed8-43b2-816a-82a83d8ddd01",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:43.681662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Threat", # conveys criticality or broad class of the event
	"EventStatus":"Active",#Active / mitigated
	"EventName":"Unexpected_Device_Connection", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Unexpected_Device_Connection:Network" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"e672ea83-2fb9-4f92-8cc8-d14f068b7d93", # globally unique, typically hash
	"DeviceName":"Router", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Network", # Samsung Galaxy 204, iPhone XXVI, etc.
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected", "Brightness": "80", "connection_strength": 12.013}
    }}

In [48]:
report_event(unexpected_connection_event)

New Threat: Unexpected_Device_Connection_345b4180-6ed8-43b2-816a-82a83d8ddd01 ThreatLevel: TL-Mid
[['ssso:Classified']]
Security Level degrades to SL-2
Record is suspended, since the current security level is not satisified.
CaptureVideo is suspended, since the current security level is not satisified.
ShareData is suspended, since the current security level is not satisified.


In [49]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv rdf:type ssso:Data. ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:PlayVideo'], ['ssso:PlayAudio'], ['ssso:StoreData']]

In [50]:
use_smart_speaker_to_record_meeting={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"Record", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:Record" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartSpeaker", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [51]:
report_event(use_smart_speaker_to_record_meeting)

Request Denied: Low SL current_sl:SL-2 user_sl:SL-4 required_sl:SL-4
SL-2 authentication is required.
Please use SmartSpeaker to perform NFC_Authentication


In [53]:
nfcauth_alice={
    "guid": "5a5437eb-4d2a-4409-abbf-760737547747", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Authentication", # conveys criticality or broad class of the event
	"EventStatus":"Success", #Success/Fail
	"EventName":"NFC_Authentication", # a type name, i.e., identifies the action, human readable
	"EventMessage":"" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea1", # globally unique, typically hash
	"DeviceName":"SmartSpeaker", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999
    }
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}
report_event(nfcauth_alice)
SmartConferenceRoom.remove((ssso['Record'], ssso['is'], ssso['Suspended']))
SmartConferenceRoom.add((ssso['Record'], ssso['is'], ssso['Active']))
SmartConferenceRoom.commit()
SmartConferenceRoom.refresh()

sl_update: Alice:SL-4->SL-4


In [54]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv rdf:type ssso:Data. ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:PlayAudio'], ['ssso:StoreData'], ['ssso:Record'], ['ssso:PlayVideo']]

In [55]:
unexpected_device_connection_mitigated={
    "guid": "7acfbfc5-83c8-414c-833a-bf432d38e335", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T21:23:52.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"2e8abc72-b5dd-4fbb-b2eb-58c2ff1926df",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:52.181662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Threat", # conveys criticality or broad class of the event
	"EventStatus":"Mitigated:345b4180-6ed8-43b2-816a-82a83d8ddd01", #Active / Mitigated
	"EventName":"Unexpected_Device_Connection", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Unexpected_Device_Connection:Network:Compromised" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"e672ea83-2fb9-4f92-8cc8-d14f068b7d93", # globally unique, typically hash
	"DeviceName":"Router", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Network", # Samsung Galaxy 204, iPhone XXVI, etc.
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected", "Brightness": "80", "connection_strength": 12.013}
    }}
SmartConferenceRoom.remove((ssso['CurrentSecurityLevel'], ssso['has'], ssso['SL-2']))
SmartConferenceRoom.add((ssso['CurrentSecurityLevel'], ssso['has'], ssso['SL-4']))
SmartConferenceRoom.commit()
SmartConferenceRoom.refresh()

In [56]:
report_event(unexpected_device_connection_mitigated)

Threat Removed: Unexpected_Device_Connection_345b4180-6ed8-43b2-816a-82a83d8ddd01
[['ssso:Classified']]
ShareData is resumed, since the threat is mitigated and current security level is satisified.
CaptureVideo is resumed, since the threat is mitigated and current security level is satisified.


In [57]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv rdf:type ssso:Data. ?serv ssso:is ssso:Active.}""")
active_service

[['ssso:PlayVideo'],
 ['ssso:Record'],
 ['ssso:CaptureVideo'],
 ['ssso:ShareData'],
 ['ssso:PlayAudio'],
 ['ssso:StoreData']]

In [58]:
unexpected_expected_door_open={
    "guid": "bfd478ac-2d27-454d-8e7d-59bf735b0844", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T21:23:42.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"1327502c-522b-413e-9bc9-ca3ee8b0ddb8",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:43.681662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Threat", # conveys criticality or broad class of the event
	"EventStatus":"Active",#Active / mitigated
	"EventName":"Unexpected_Door_Open", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Unexpected_Door_Open:Compromised:Environment" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"e672ea83-2fb9-4f92-8cc8-d14f068b7d93", # globally unique, typically hash
	"DeviceName":"SmartDoorLock", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"Lock", # Samsung Galaxy 204, iPhone XXVI, etc.
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected", "connection_strength": 12.013}
    }}

In [59]:
report_event(unexpected_expected_door_open)

New Threat: Unexpected_Door_Open_1327502c-522b-413e-9bc9-ca3ee8b0ddb8 ThreatLevel: TL-High
[['ssso:Classified']]
Security Level degrades to SL-0
PlayAudio is suspended, since the current security level is not satisified.
ShareData is suspended, since the current security level is not satisified.
Record is suspended, since the current security level is not satisified.
PlayVideo is suspended, since the current security level is not satisified.
CaptureVideo is suspended, since the current security level is not satisified.
StoreData is suspended, since the current security level is not satisified.


In [60]:
use_projector_to_play_video={
    "guid": "463f652c-7eac-455d-847c-3017a3ac4e48", # allows distinguishing simultaneous events
    "timestamp":"2018-07-03T22:23:53.581662+00:00", # ISO in UTC, but could be Unix, not unique
    "event":{ 
	# Not every event contains non-NULL values for each of these
	# but the items *should* be filled with NULL for simplicity of parsing
	"EventID":"386388f3-481d-4d90-9b9d-eb6ae62cbdc6",  # globally unique id
	"EventTimestamp":"2018-07-03T21:23:22.481662+00:00", #in the case the event time != report timestamp, not unique
	"EventType":"Request", # conveys criticality or broad class of the event
	"EventStatus":"Request", #Request/ Disable
	"EventName":"PlayVideo", # a type name, i.e., identifies the action, human readable
	"EventMessage":"Request:PlayVideo" # for future use, not generated yet (description of event)
    },

    "device":
    {
	"DeviceID":"63da70b3-06d8-4689-9d94-3aea15938ea3", # globally unique, typically hash
	"DeviceName":"Projector", # conveys identification to user/admin, human readable
	"DeviceLocation":"BLD4-F16-M201-2", # human readable description of abstract location
	"DeviceTypeName":"AV", 
	"DeviceGroup":["Secure_Smart_Conference_Room_V1"], # list of administrative groups from UEM or other manager

	"DeviceAttributes":{ # colour commentary
	    "attributes1":"",
	    "attributes2":"",
	},

	# status of the individual modules as a list of pairs of values
	#INCLUDES the tagging or other information about attack and its impact on the data
	"Status": {"wifi":"Connected",  "connection_strength": 10.999}
    },
    "User":{"UserName":"Alice",
            "UID":"10001"
           }
}

In [61]:
report_event(use_projector_to_play_video)

Request Denied: Low SL current_sl:SL-0 user_sl:SL-4 required_sl:SL-2
SL-2 authentication is required.
Please use SmartSpeaker to perform Voice_Authentication


In [ ]:
event_name=event["event"]["EventName"]
ql="""select ?t where{ssso:%s ssso:is ?t.}"""%event_name

In [ ]:
ql

In [ ]:
SmartConferenceRoom.query(PREFIX+ql)[0][0]=="ssso:Template"

In [ ]:
event_id=event["event"]["EventID"]

In [ ]:
event_id

In [ ]:
ssso[event_name+'_'+event_id]

In [ ]:
SmartConferenceRoom.query(PREFIX+"""select ?tl where{?tl rdf:type ssso:ThreatLevel. ssso:%s ssso:has ?tl.}"""%event_name)[0][0].split(':')[-1]

In [ ]:
device_name=event["device"]["DeviceName"]
device_name

In [ ]:
q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ssso: <http://www.semanticweb.org/linch/ontologies/2018/11/ssso#>
select ?t where {
?t rdf:type ssso:Threat.
?t ssso:is ssso:Active.
}
"""
SmartConferenceRoom.query(q)

In [ ]:
SmartConferenceRoom.remove((ssso['Unexpected_Occupancy_82b69dc9-cdbb-4637-ac7a-2141b4fee214'],None,None))

In [ ]:
event_name+'_'+event_id

In [ ]:
q = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX ssso: <http://www.semanticweb.org/linch/ontologies/2018/11/ssso#>
select ?tl where {
?tl rdf:type ssso:ThreatLevel.
ssso:Unexpected_Occupancy_82b69dc9-cdbb-4637-ac7a-2141b4fee214 ssso:has ?tl
}
"""
SmartConferenceRoom.query(q)


In [ ]:
SmartConferenceRoom.add((ssso[event_name+'_'+event_id], RDF.type, ssso['Threat']))

In [ ]:
tl=SmartConferenceRoom.query(PREFIX+"""select ?tl where{?tl rdf:type ssso:ThreatLevel. ssso:%s ssso:has ?tl.}"""%event_name)[0][0].split(':')[-1]
SmartConferenceRoom.add((ssso[event_name+'_'+event_id], ssso['has'], ssso[tl]))

In [ ]:
SmartConferenceRoom.add((ssso[event_name+'_'+event_id], ssso['is'], ssso['Active'])) #Status: Active Threat

In [ ]:
event_id.replace('-','_')

In [ ]:
SmartConferenceRoom.query(PREFIX+ql)

In [ ]:
event_status=occupancy_camera_event_mitigated["event"]["EventStatus"]
ql="""select ?t where{ssso:%s ssso:is ?t.}"""%(event_name+event_status.split(":")[1])
res=SmartConferenceRoom.query(PREFIX+ql)
res

In [ ]:
event_name=occupancy_camera_event_mitigated["event"]["EventName"]
event_status=occupancy_camera_event_mitigated["event"]["EventStatus"]
event_name+event_status.split(":")[1]

In [ ]:
ql="""select ?t where{?t rdf:type ssso:Threat.}"""
res=SmartConferenceRoom.query(PREFIX+ql)
res

In [ ]:
SmartConferenceRoom.remove((ssso['Unexpected_Occupancy_82b69dc9-cdbb-4637-ac7a-2141b4fee214'],None,None))

In [ ]:
SmartConferenceRoom.refresh()

In [ ]:
ql="""select ?t where{?t rdf:type ssso:Status. ssso:%s ssso:is ?t.}"""%irisauth_alice["event"]["EventName"]
res=SmartConferenceRoom.query(PREFIX+ql)
res
sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:provides ?sl.}"""%irisauth_alice["event"]["EventName"])[0][0].split(':')[-1]
sl

In [ ]:
uid='10001'
name=("""select ?u where{?u rdf:type ssso:User. ?u ssso:has ssso:%s.}"""%uid)[0][0].split(':')[-1]
name

In [ ]:
name=SmartConferenceRoom.query((PREFIX+"""select ?n where{?n rdf:type ssso:User. ?n ssso:has ssso:%s .}"""%uid))[0][0].split(':')[-1]

name

In [ ]:
PREFIX+"""select ?n where{?n rdf:type ssso:User. ?n ssso:has ssso:%s.}"""%uid

In [ ]:
username="Alice"
usl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:has ?sl.}"""%username)[0][0].split(':')[-1]
usl

In [ ]:
"1%s2%s"%(1,2)

In [ ]:
sl_update('Alice','SL-4')

In [ ]:
SmartConferenceRoom.remove((ssso['Alice'],ssso['has'],ssso['SL-4']))
SmartConferenceRoom.add((ssso['Alice'],ssso['has'],ssso['SL-0']))

In [ ]:
irisauth_alice.keys()

In [ ]:
prev_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:has ?sl.}"""%'Alice')[0][0].split(':')[-1]

In [ ]:
prev_sl

In [ ]:
servicename='OpenDoor'
ql="""select ?sl where{?sl rdf:type ssso:SecurityLevel. ssso:%s ssso:requires ?sl.}"""%servicename
res=SmartConferenceRoom.query(PREFIX+ql)
res

In [ ]:
current_sl=SmartConferenceRoom.query(PREFIX+"""select ?sl where{ssso:CurrentSecurityLevel ssso:has ?sl.}""")
current_sl

In [ ]:
int('SL-4'[-1])-int('SL-2'[-1])

In [ ]:
de=SmartConferenceRoom.query(PREFIX+"""select ?e where{?e rdf:SubClassOf ssso:Equipment.}""")
de

In [ ]:
ql="""select ?st where{?st rdf:type ssso:Status. ssso:%s ssso:is ?st.}"""%'SmartDoorLock'
res=SmartConferenceRoom.query(PREFIX+ql)
res[0][0].split(':')[-1]

In [ ]:
sl_diff('SL-2','SL-4')

In [ ]:
max('SL-2','SL-3')

In [ ]:
ql="""select ?as where{?as rdf:type ssso:Authentication. ?as ssso:provides ssso:%s. ssso:%s ssso:provides ?as.}"""%('SL-2','SmartDoorLock')
res=SmartConferenceRoom.query(PREFIX+ql)
res

In [ ]:
import random

In [ ]:
random.sample(res,1)

In [ ]:
prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ssso:%s ssso:isProvidedBy ?au.}"""%'Alice')
prev_auth

In [ ]:
prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ssso:%s ssso:isProvidedBy ?au.}"""%'Bob')
'Face_' in prev_auth[0][0]

In [ ]:
SmartConferenceRoom.add((ssso['Bob'],ssso['has'],ssso['SL-0']))

In [ ]:
SmartConferenceRoom.remove((ssso['Bob'],ssso['isProvidedBy'],ssso['Face_Authentication']))

In [ ]:
SmartConferenceRoom.commit()

In [ ]:
SmartConferenceRoom.refresh()

In [ ]:
obj='SmartDoorLock'
prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ssso:%s ssso:provides ?au. ?au ssso:provides ssso:%s}"""%(obj,'SL-1'))
prev_auth

In [ ]:
obj='SmartDoorLock'
prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s. ssso:%s ssso:provides ?au. ssso:%s ssso:is ssso:Enabled.}"""%('SL-1',obj,obj))
prev_auth

In [ ]:
sl_satisfied_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s.}"""%('SL-1'))
sl_satisfied_auth

In [ ]:
usr_prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ssso:%s ssso:isProvidedBy ?au.}"""%'Bob')
usr_prev_auth

In [ ]:
sl_satisfied_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s.}"""%('SL-1'))
sl_satisfied_auth

In [ ]:
for i in range(len(current_device_auth)):
    for j in range(len(usr_prev_auth)):
        if(usr_prev_auth[j][0]==current_device_auth[i][0]):
            current_device_auth.remove([usr_prev_auth[j][0]])
for i in range(len(sl_satisfied_auth)):
    for j in range(len(usr_prev_auth)):
        if(usr_prev_auth[j][0]==sl_satisfied_auth[i][0]):
            sl_satisfied_auth.remove([usr_prev_auth[j][0]])

In [ ]:
sl_satisfied_auth.remove(['ssso:NFC_Authentication'])

In [ ]:
random.sample(sl_satisfied_auth,1)[0][0].split(':')[-1]

In [ ]:
au=random.sample(sl_satisfied_auth,1)[0][0].split(':')[-1]
au

In [ ]:
dev=SmartConferenceRoom.query(PREFIX+"""select ?dev where{?dev ssso:is ssso:Enabled. ?dev ssso:provides ssso:%s.}"""%(au))
random.sample(dev,1)[0][0].split(':')[-1]

In [ ]:
current_device_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s. ssso:%s ssso:provides ?au. ssso:%s ssso:is ssso:Enabled.}"""%('SL-2','SmartDoorLock','SmartDoorLock'))
current_device_auth

In [ ]:
usr_prev_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ssso:%s ssso:isProvidedBy ?au.}"""%'Carol')
usr_prev_auth

In [ ]:
for i in range(len(current_device_auth)):
    for j in range(len(usr_prev_auth)):
        print(i,j)
        if(usr_prev_auth[j][0]==current_device_auth[i][0]):
             current_device_auth[j]=['Invalid']
current_device_auth.remove(['Invalid'])

In [ ]:
usr_prev_auth[0][0]

In [ ]:
current_device_auth[0][0]

In [ ]:
current_device_auth.remove(['Invalid'])

In [ ]:
current_device_auth

In [ ]:
servicename='Record'
service_category=SmartConferenceRoom.query(PREFIX+"""select ?sc where{?sc rdfs:subClassOf ssso:Service. ssso:%s rdf:type ?sc.}"""%servicename)
service_category

In [ ]:
['ssso:Sense'] in service_category

In [ ]:
sldegradation('SL-4','TL-Low')

In [ ]:
tl=SmartConferenceRoom.query(PREFIX+"""select ?tl where {?tr rdf:type ssso:Threat. ?tl rdf:type ssso:ThreatLevel.?tr ssso:is ssso:Active. ?tr ssso:has ?tl}""")
tl

In [ ]:
max('TL-High','TL-Mid')

In [ ]:
max_tl(tl)

In [ ]:
ql="""select ?t where{ssso:%s ssso:has ?t.}"""%'Unexpected_Device_Connection'
res=SmartConferenceRoom.query(PREFIX+ql)
res

In [ ]:
active_service=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?serv rdf:type ssso:Data. ?serv ssso:is ssso:Active.}""")
active_service

In [ ]:
active_service2=SmartConferenceRoom.query(PREFIX+"""select ?serv where{ ?t rdfs:subClassOf ssso:Service. ?serv rdf:type ?t. ?serv ssso:is ssso:Active.}""")
active_service2

In [ ]:
sl_satisfied_auth=SmartConferenceRoom.query(PREFIX+"""select ?au where{?au rdf:type ssso:Authentication. ?au ssso:provides ssso:%s. }"""%('SL-2'))
sl_satisfied_auth